In [1]:
from ipynb.fs.defs.utils import read_lines

In [2]:
import re

In [3]:
puzzle_input = read_lines('day12.txt')

In [4]:
test_input = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""".splitlines()

In [5]:
OK = '.'
NOK = '#'
UNKNOWN = '?'

In [6]:
def part1(inp):
    records = parse_input(inp)
    return sum([len(get_arrangements(rec[0], rec[1])) for rec in records])

In [7]:
def parse_input(inp):
    return [parse_record(rec) for rec in inp]

In [8]:
def parse_record(record):
    schema, groups = record.split(' ')
    return (schema, tuple(map(int, groups.split(',')))) # using tuple because it's hashable contrary to list

In [9]:
def get_arrangements(schema, groups):
    # print(f'inp: {schema}, {groups}')
    if len(groups) < 1:
        if not NOK in schema:
            print(f'found! leftover: {schema}')
        return [] if NOK in schema else [OK*len(schema)]
    options = []
    group = groups[0]
    last_idx_with_enough_space = len(schema) - sum(groups) + 1 
    first_NOK = schema.find(NOK)
    must_match_until = min(first_NOK + 1, last_idx_with_enough_space) if first_NOK >= 0 else last_idx_with_enough_space
    for i in range(must_match_until):
        #print(f'try {schema[i:i+group]}, groups {groups}')
        if len(groups) == 1:
            if valid_group(schema[i:i+group]) and not NOK in schema[i+group:]:
                options.append(OK*i + NOK*group + OK*(len(schema) - i - group))
        else:
            if valid_group(schema[i:i+group]) and valid_separator(schema[i+group]):
                #print(f'valid: {schema[i:i+group]}, {groups}, {i}')
                option_start = OK*i + NOK*group + OK
                options += [option_start + option_end for option_end in get_arrangements(schema[i+group+1:], groups[1:])]
    #print(f'{schema}, {groups} ---> {options}')
    return options

In [10]:
def valid_group(schema):
    return not OK in schema

In [11]:
def valid_separator(char):
    return char != NOK

In [12]:
get_arrangements(*parse_record('?###???????? 3,2,1'))

['.###.##.#...',
 '.###.##..#..',
 '.###.##...#.',
 '.###.##....#',
 '.###..##.#..',
 '.###..##..#.',
 '.###..##...#',
 '.###...##.#.',
 '.###...##..#',
 '.###....##.#']

In [13]:
get_arrangements(*parse_record('??? 1'))

['#..', '.#.', '..#']

In [14]:
part1(puzzle_input)

7032

In [15]:
def part2(inp):
    records = parse_input(inp)
    expanded = [ ( UNKNOWN.join([rec[0]] * 5), rec[1] * 5 ) for rec in records]
    return sum([count_arrangements(rec[0], rec[1]) for rec in expanded])

In [16]:
cache = {}
def count_arrangements(schema, groups):
    if (schema, groups) in cache:
        return cache[(schema, groups)]
    options = 0
    group = groups[0]
    last_idx_with_enough_space = len(schema) - sum(groups) + 1 
    first_NOK = schema.find(NOK)
    must_match_until = min(first_NOK + 1, last_idx_with_enough_space) if first_NOK >= 0 else last_idx_with_enough_space
    for i in range(must_match_until):
        if len(groups) == 1:
            if valid_group(schema[i:i+group]) and not NOK in schema[i+group:]:
                options += 1
        else:
            if valid_group(schema[i:i+group]) and valid_separator(schema[i+group]):
                options += count_arrangements(schema[i+group+1:], groups[1:])
    #print(f'{schema}, {groups} ---> {options}')
    cache[(schema, groups)] = options
    return options

In [17]:
part2(puzzle_input)

1493340882140

In [18]:
len(cache)

119962